# 人脸检测和识别推理流程

以下示例展示了如何使用facenet_pytorch python包，在使用在VGGFace2数据集上预训练的Inception Resnet V1模型上对图像数据集执行人脸检测和识别。

以下PyTorch方法已包含：

* 数据集
* 数据加载器
* GPU/CPU处理

In [13]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

#### 判断是否有nvidia GPU可用

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('在该设备上运行: {}'.format(device))

在该设备上运行: cpu


#### 定义MTCNN模块

为了说明，默认参数已显示，但不是必需的。请注意，由于MTCNN是一组神经网络和其他代码，因此必须以以下方式传递设备，以便在需要内部复制对象时启用。

查看`help(MTCNN)`获取更多细节。

In [3]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### 定义Inception Resnet V1模块

设置classify=True以使用预训练分类器。对于本示例，我们将使用该模型输出嵌入/卷积特征。请注意，在推理过程中，将模型设置为`eval`模式非常重要。

查看`help(InceptionResnetV1)`获取更多细节。

In [4]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

#### 定义数据集和数据加载器

我们向数据集添加了`idx_to_class`属性，以便稍后轻松重编标签索引为身份名称。

In [5]:
# 返回批次的第一个样本
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('./data/test_images') ##. 表示同级目录
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()} # 创建字典表示id 和类名的对应关系
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

#### 执行MTCNN人脸检测

迭代DataLoader对象并检测每个人脸及其关联的检测概率。如果检测到脸部，`MTCNN`的前向方法将返回裁剪到检测到的脸部的图像。默认情况下，仅返回检测到的单个面部-要使`MTCNN`返回所有检测到的面部，请在上面创建MTCNN对象时设置`keep_all=True`。

要获取边界框而不是裁剪的人脸图像，可以调用较低级别的`mtcnn.detect()`函数。查看`help(mtcnn.detect)`获取详细信息。

In [6]:
aligned = [] # 存储检测到的人脸
names = [] 
# x: 图片数据
# y: 图片对应的标签
for x, y in loader:
    print(x,y)
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('检测到的人脸及其概率: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

<PIL.Image.Image image mode=RGB size=1665x2048 at 0x2716FEB4320> 0
检测到的人脸及其概率: 0.999983
<PIL.Image.Image image mode=RGB size=2341x3600 at 0x271647C8C20> 1
检测到的人脸及其概率: 0.999934
<PIL.Image.Image image mode=RGB size=1125x1612 at 0x2716FF65430> 2
检测到的人脸及其概率: 0.999733
<PIL.Image.Image image mode=RGB size=1638x2048 at 0x2716FB74140> 3
检测到的人脸及其概率: 0.999880
<PIL.Image.Image image mode=RGB size=2048x1991 at 0x2716FF65490> 4
检测到的人脸及其概率: 0.999992
<PIL.Image.Image image mode=RGB size=2102x3000 at 0x2716FEB5FD0> 5
检测到的人脸及其概率: 0.999597
<PIL.Image.Image image mode=RGB size=678x1206 at 0x2716FC23E90> 6
检测到的人脸及其概率: 1.000000


In [7]:
# from PIL import Image

# # 遍历 aligned 列表中的图片
# for i, face in enumerate(aligned):
#     # 转换为 NumPy 数组并转换为 PIL 图像
#     face_image = Image.fromarray((face.permute(1, 2, 0).cpu().numpy() * 255).astype('uint8'))
#     face_image.show(title=f"Aligned Face {i+1}")

#### 计算图像嵌入

MTCNN将返回所有面部图像的相同大小，从而可以使用Resnet识别模块轻松进行批处理。在这里，由于我们只有一些图像，因此我们构建一个单个批次并对其执行推理。

对于实际数据集，代码应修改为控制传递给Resnet的批处理大小，特别是如果在GPU上处理。对于重复测试，最好将人脸检测（使用MTCNN）与嵌入或分类（使用InceptionResnetV1）分开，因为剪切面或边界框的计算可以一次执行，检测到的面部可以保存供将来使用。

In [8]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [9]:
from sklearn.cluster import KMeans
import numpy as np

# 假设 embeddings 是一个 PyTorch 张量，先转换为 NumPy 数组
embeddings_np = embeddings.numpy()

# 设置聚类的类别数量（例如 3 类）
num_clusters = 6

# 使用 K-Means 聚类
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(embeddings_np)

# 获取每个嵌入向量的聚类标签
labels = kmeans.labels_

# 打印聚类结果
for i, label in enumerate(labels):
    print(f"Embedding {i} belongs to cluster {label}")

Embedding 0 belongs to cluster 5
Embedding 1 belongs to cluster 2
Embedding 2 belongs to cluster 0
Embedding 3 belongs to cluster 4
Embedding 4 belongs to cluster 3
Embedding 5 belongs to cluster 1
Embedding 6 belongs to cluster 1


d:\software\anaconda\V_environment\Pytorch\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


#### 打印各类别的距离矩阵

In [10]:
# dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
# pd.DataFrame(dists, columns=names, index=names)

# 获取最后一个嵌入向量
last_embedding = embeddings[-1]

# 计算最后一个嵌入向量与前几个嵌入向量的欧几里得距离
dists = [(last_embedding - e).norm().item() for e in embeddings[:-1]]

# 打印结果
index = np.argmin(dists)   
print("the picture is " + names[index])

the picture is yhj


In [11]:
# from PIL import Image, ImageDraw
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

In [12]:
# image_path = 'D:/courses/software/team-project-25spring-g10/AI/human_face/data/multiface.jpg'
# image = Image.open(image_path)
# image = image.convert('RGB')

# # 检测人脸
# boxes, _ = mtcnn.detect(image)

# # 绘制人脸框
# image_draw = image.copy()
# draw = ImageDraw.Draw(image_draw)
# if boxes is not None:
#     for box in boxes:
#         draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)

